In [15]:
import numpy as np
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt

In [98]:
def load_dataset(name):
    path = "Datasets/{}".format(name)
    with open(path) as f:
        arr = []
        for line in f:
            line = line.strip()
            arr.append(line.split("\t"))
    D = np.array(arr, dtype=np.float32)
    Y = D[:,0].reshape(-1,1)
    y = (Y == 1) * 1 + (Y != 1) * -1
    return (D[:,1:], y)

In [153]:
def make_adj(X):
    W = np.zeros((X.shape[0], X.shape[0]))
    kd = KDTree(X)
    idx = kd.query(X, k=3, return_distance=False)
    for i in range(idx.shape[0]):
        for j in idx[i]:
            W[i,j] = (j != i) * 1
            W[j,i] = W[i,j]
    D = np.diag(np.sum(W, axis=1).T)
    return W, D, (D - W)

def harmonic(X, y, l):
    l_end = len(l)
    all_idx = np.arange(start=0, stop=y.shape[0], step=1, dtype=np.uint32)
    l_msk = np.ones(y.shape[0], dtype=bool)
    l_msk[l] = False
    u = all_idx[l_msk]
    X_ = np.vstack([X[l,:], X[u,:]])
    y_ = np.vstack([y[l,:], y[u,:]])
    W,_,L = make_adj(X_)
    L_inv = np.linalg.pinv(L[l_end:, l_end:])
    interploated = L_inv @ W[l_end:, :l_end] @ y_[:l_end]
    return y_, np.vstack([y_[:l_end], np.sign(interploated)])

def harmonic_kernel(X, y, l):
    _,_,L = make_adj(X)
    l_len = len(l)
    L_inv = np.linalg.pinv(L)
    K = np.zeros((l_len, l_len))
    for i in range(l_len):
        for j in range(l_len):
            K[i, j] = L_inv[l[i],l[j]]
    y_l = y[l]
    alpha = np.linalg.pinv(K) @ y_l
    ae = np.zeros((1, L.shape[0]))
    for i, idx in enumerate(l):
        ae[0, idx] = alpha[i]
    return y, np.sign(ae @ L_inv).T

def sample_labels(y, num):
    return np.random.choice(y, size=(num,))

def emp_err(gt, pred):
    return (gt != pred).astype(int).sum() / len(gt)

def laplacian_introp(X, y, sample_idx):
    sz = len(sample_idx)
    gt_all, pred_all = harmonic(X, y, sample_idx)
    return emp_err(gt_all[sz:], pred_all[sz:])

def laplacian_kernel_introp(X, y, sample_idx):
    gt_all, pred_all = harmonic_kernel(X, y, sample_idx)
    return emp_err(gt_all, pred_all)
    

In [154]:
def run_protocol():
    ds_size = [50, 100, 200, 400]
    l_size = [1,2,4,8,16]
    tables = np.zeros((len(ds_size), len(l_size), 2))
    for i, sz in enumerate(ds_size):
        X, y = load_dataset("dtrain13_{}.dat".format(sz))
        clss_1 = np.arange(0, sz, 1, int)
        clss_2 = np.arange(sz, sz * 2, 1, int)
        for j, l in enumerate(l_size):
            errLI = 0
            errKLI = 0
            for _ in range(20):
                sampled_clss_1 = np.random.choice(clss_1, (l,), replace=False)
                sampled_clss_2 = np.random.choice(clss_2, (l,), replace=False)
                L_cal = np.hstack([sampled_clss_1, sampled_clss_2])
                errLI += laplacian_introp(X, y, L_cal)
                errKLI += laplacian_kernel_introp(X, y, L_cal)
            tables[i, j, 0] = errLI / 20
            tables[i, j, 1] = errKLI / 20
            print("size={}, l={}: eli={}, ekli={}".format(sz, l, tables[i, j, 0], tables[i, j, 1]))
    return tables

In [152]:
tables = run_protocol()

size=50, l=1: eli=0.18061224489795916, ekli=0.057499999999999996
size=50, l=2: eli=0.134375, ekli=0.06600000000000002
size=50, l=4: eli=0.07228260869565217, ekli=0.06600000000000003
size=50, l=8: eli=0.050595238095238096, ekli=0.04
size=50, l=16: eli=0.05220588235294117, ekli=0.026500000000000003
size=100, l=1: eli=0.12979797979797975, ekli=0.10725
size=100, l=2: eli=0.07066326530612244, ekli=0.06275
size=100, l=4: eli=0.06848958333333333, ekli=0.061750000000000006
size=100, l=8: eli=0.052445652173913046, ekli=0.048750000000000016
size=100, l=16: eli=0.042261904761904764, ekli=0.03425000000000001
size=200, l=1: eli=0.11608040201005025, ekli=0.018125000000000002
size=200, l=2: eli=0.05694444444444445, ekli=0.01987500000000001
size=200, l=4: eli=0.02844387755102041, ekli=0.021500000000000005
size=200, l=8: eli=0.017838541666666666, ekli=0.018000000000000006
size=200, l=16: eli=0.01807065217391304, ekli=0.016125000000000004
size=400, l=1: eli=0.12349624060150374, ekli=0.011062500000000003

In [150]:
tables[:,:,0]

array([[0.23469388, 0.15104167, 0.09836957, 0.05654762, 0.05220588],
       [0.19545455, 0.06862245, 0.0765625 , 0.06657609, 0.03630952],
       [0.07248744, 0.04267677, 0.02295918, 0.02083333, 0.01970109],
       [0.1127193 , 0.01991206, 0.01559343, 0.01619898, 0.0140625 ]])

In [151]:
tables[:,:,1]

array([[0.078    , 0.074    , 0.058    , 0.0435   , 0.031    ],
       [0.17275  , 0.06125  , 0.064    , 0.0525   , 0.0305   ],
       [0.016    , 0.017125 , 0.0165   , 0.0185   , 0.017    ],
       [0.01     , 0.014375 , 0.0086875, 0.0100625, 0.00925  ]])